### Question Answer Application
The goal of Question Answering is to find the answer to a question given a question and an accompanying context. The predicted answer will be either a span of text from the context or an empty string (indicating the question cannot be answered from the context).

In [85]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:

In [ ]:
!pip install transformers datasets evaluate

In [13]:
from huggingface_hub import notebook_login
from datasets import load_dataset
notebook_login()

In [217]:
from datasets import load_dataset
squad = load_dataset("squad", split="train[:10000]")

In [218]:
squad = squad.train_test_split(test_size=0.2)

In [219]:
squad["train"]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 8000
})

In [220]:
def convert_dataset(dataset):
    sub = {}
    sub['context'] = dataset['context']
    sub['qas'] = [{}]
    sub['qas'][0]['id'] = dataset['id']
    answers = dataset['answers']['text']
    # print(answers)
    if len(answers) > 0 :
      sub['qas'][0]['is_impossible'] = False
    else:
      sub['qas'][0]['is_impossible'] = True
    sub['qas'][0]['question'] = dataset['question']
    sub['qas'][0]['answers'] = [{}]
    sub['qas'][0]['answers'][0]['text'] = ' '.join(answers)
    sub['qas'][0]['answers'][0]['answer_start'] = dataset['answers']['answer_start'][0]

    return sub

In [221]:
train = []
for id in range(8000):
  ret = convert_dataset(squad['train'][id])
  train.append(ret)

In [222]:
train[0]

{'context': 'The coverage of the events by the media came under scrutiny during the relay. Chinese media coverage of the torch relay has been distinct in a number of ways from coverage elsewhere. Western reporters in Beijing have described Chinese media coverage as partial and censored (for example when Chinese media did not broadcast Reporters Without Borders\' disruption of the torch lighting ceremony), whereas Chinese netizens have in turn accused Western media coverage of being biased. The French newspaper Libération was criticised by the Chinese State press agency Xinhua for its allegedly biased reporting; Xinhua suggested that Libération needed "a stinging slap in the face" for having "insulted the Olympic flame" and "supported a handful of saboteurs".',
 'qas': [{'id': '56db7278e7c41114004b5127',
   'is_impossible': False,
   'question': 'The Chinese State Press said the French newspaper was supporting a handful of what?',
   'answers': [{'text': 'saboteurs', 'answer_start': 743

In [223]:
test = []
for id in range(2000):
  ret = convert_dataset(squad['test'][id])
  test.append(ret)

In [224]:
test[0]

{'context': "To Kill a Mockingbird is a novel by Harper Lee published in 1960. It was immediately successful, winning the Pulitzer Prize, and has become a classic of modern American literature. The plot and characters are loosely based on the author's observations of her family and neighbors, as well as on an event that occurred near her hometown in 1936, when she was 10 years old.",
 'qas': [{'id': '56ce30f5aab44d1400b88521',
   'is_impossible': False,
   'question': 'What prize did To Kill a Mockingbird win?',
   'answers': [{'text': 'Pulitzer Prize', 'answer_start': 109}]}]}

In [142]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [143]:
model_type="xlnet"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [144]:
# Configure the model
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5


In [148]:
# ! pip install wandb

# Bert

In [168]:
model_type="bert"
model_name= "bert-base-cased"

In [173]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "use_early_stopping": True,
    'early_stopping_consider_epochs': False,
    'early_stopping_delta': 0,
    # 'early_stopping_metric': f"str = {'correct'}",
    'early_stopping_metric_minimize': False,
    'early_stopping_patience': 2,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }

}

In [174]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [175]:
### Remove output folder
!rm -rf outputs

In [176]:
# Train the model
model.train_model(train, eval_data=test)

add example index and unique id: 100%|██████████| 3200/3200 [00:00<00:00, 514974.21it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

correct,▁
eval_loss,▁
global_step,▁
incorrect,▁
similar,▁
train_loss,▁
correct,17
eval_loss,-0.88364
global_step,26
incorrect,101
similar,682


Running Epoch 0 of 5:   0%|          | 0/26 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

...

Running Epoch 3 of 5:   0%|          | 0/26 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/26 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

(130,
 {'global_step': [26, 52, 78, 104, 130],
  'correct': [55, 285, 311, 322, 328],
  'similar': [670, 328, 349, 351, 325],
  'incorrect': [75, 187, 140, 127, 147],
  'train_loss': [2.854418992996216,
   1.709473729133606,
   1.0184025764465332,
   0.6743239164352417,
   0.9031333923339844],
  'eval_loss': [-1.4574068509615385,
   -2.8455528846153846,
   -3.7295673076923075,
   -4.431189903846154,
   -4.611778846153846]})

In [177]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [178]:
texts

{'correct_text': {'56d4bc642ccc5a1400d83191': '541,000',
  '56cf69144df3c31400b0d746': 'Feliks Jarocki',
  '56d51b832593cc1400307a79': 'fewer than 60',
  '5734227dd058e614000b69bd': '94.8 percent',
  '56d381b459d6e41400146595': 'June 1849',
  '56d624261c85041400946f2d': 'large landslides',
  '56ce3406aab44d1400b8856c': '1521–1567',
  '56cd727662d2951400fa65bd': 'March 10, 1403',
  '56cee638aab44d1400b88c31': 'Fernando Wood',
  '5731c2fd0fdd8d15006c6518': 'fungi',
  '56beb2a43aeaaa14008c9239': 'Freddie Gray',
  '56d523bd2593cc1400307aaa': 'Tulong reservoir',
  '56be8a583aeaaa14008c9095': 'Darlette Johnson',
  '56cd798262d2951400fa65f7': 'Nanjing',
  '56cdce0b62d2951400fa6858': 'April 30, 2008',
  '56d4d2f12ccc5a1400d8326c': 'America the Beautiful',
  '56cd8d2162d2951400fa66ce': '1415',
  '56d484312ccc5a1400d83161': 'The Fighting Temptations',
  '56cc07886d243a140015ee64': 'London',
  '56ce1225aab44d1400b88432': '22 February 1810',
  '56beca973aeaaa14008c9478': 'two',
  '56cf4815aab44d14

In [179]:
context = """Cristiano Ronaldo dos Santos Aveiro GOIH ComM (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Professional League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards[note 3] and four European Golden Shoes, the most by a European player. He has won 32 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42) in the Champions League, goals in the European Championship (14), international goals (123) and international appearances (200). He is one of the few players to have made over 1,100 professional career appearances, and has scored over 800 official senior career goals for club and country, making him the highest goalscorer of all time.

Ronaldo began his senior career with Sporting CP, before signing with Manchester United in 2003, winning the FA Cup in his first season. He would also go on to win three consecutive Premier League titles, the Champions League and the FIFA Club World Cup; at age 23, he won his first Ballon d'Or. Ronaldo was the subject of the then-most expensive association football transfer when he signed for Real Madrid in 2009 in a transfer worth €94 million (£80 million). He became a key contributor and formed an attacking trio with Karim Benzema and Gareth Bale which was integral to the team winning four Champions League wins from 2014 to 2018, including La Décima. During this period, he won back-to-back Ballons d'Or in 2013 and 2014, and again in 2016 and 2017, and was runner-up three times behind Lionel Messi, his perceived career rival. He also became the club's all-time top goalscorer and the all-time top scorer in the Champions League, and finished as the competition's top scorer for six consecutive seasons between 2012 and 2018. With Real, Ronaldo won four Champions Leagues, two La Liga titles, two Copas del Rey, three UEFA Super Cups and three Club World Cups. In 2018, he signed for Juventus in a transfer worth an initial €100 million (£88 million), the most expensive transfer for an Italian club and for a player over 30 years old. He won two Serie A titles, two Supercoppa Italiana trophies and a Coppa Italia, became the inaugural Serie A Most Valuable Player and became the first footballer to finish as top scorer in the English, Spanish and Italian leagues. He returned to Manchester United in 2021, finishing his only full season as the club's top scorer, before his contract was terminated in 2022. In 2023, he signed for Al Nassr.

Ronaldo made his international debut for Portugal in 2003 at the age of 18 and has since earned 200 caps, making him Portugal's most-capped player. With more than 100 goals at international level, he is also the all-time top goalscorer. Ronaldo has played in and scored at eleven major tournaments; he scored his first international goal at Euro 2004, where he helped Portugal reach the final. He assumed captaincy of the national team in July 2008. In 2015, Ronaldo was named the best Portuguese player of all time by the Portuguese Football Federation. The following year, he led Portugal to their first major tournament title at Euro 2016, and received the Silver Boot as the second-highest goalscorer of the tournament. This achievement would see him receive his fourth Ballon d'Or. He also led them to victory in the inaugural UEFA Nations League in 2019, receiving the top scorer award in the finals, and later received the Golden Boot as top scorer of Euro 2020.

One of the world's most marketable and famous athletes, Ronaldo was ranked the world's highest-paid athlete by Forbes in 2016 and 2017 and the world's most famous athlete by ESPN from 2016 to 2019. Time included him on their list of the 100 most influential people in the world in 2014. He is the first footballer and the third sportsman to earn US$1 billion in his career."""

In [182]:
# Make predictions with the model
to_predict = [
    {
        "context": context,
        "qas": [
            {
                "question": "Where did ronaldo started his career",
                "id": "0",
            }
        ],
    }
]

In [183]:
answers, probabilities = model.predict(to_predict)

print(answers), probabilities

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3175.10it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['', 'Ronaldo won four Champions Leagues, two La Liga titles, two Copas del Rey, three UEFA Super Cups and three Club World Cups. In 2018, he signed for Juventus']}]


(None,
 [{'id': '0', 'probability': [0.998948573064696, 0.0005726915414937041]}])

# distilbert-base-cased

In [192]:
model_type == "distilbert"
model_name = "distilbert-base-cased"

In [193]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "use_early_stopping": True,
    'early_stopping_consider_epochs': False,
    'early_stopping_delta': 0,
    # 'early_stopping_metric': f"str = {'correct'}",
    'early_stopping_metric_minimize': False,
    'early_stopping_patience': 2,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }

}

In [194]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['encoder.layer.9.output.dense.weight', 'encoder.layer.1.attention.output.dense.weight', 'embeddings.LayerNorm.weight', 'encoder.layer.6.attention.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.8.output.dense.bias', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.1.attention.self.query.weight', 'encoder.layer.1.attention.self.query.bias', 'encoder.layer.3.attention.output.LayerNorm.bias', 'encoder.layer.2.attention.self.key.bias', 'embeddings.word_embeddings.weight', 'encoder.layer.5.output.LayerNorm.weight', 'encoder.layer.9.attention.self.key.weight', 'encoder.layer.9.attention.self.key.bias', 'encoder.layer.9.attention.output.dense.weight', 'encoder.layer.5.attention.output.dense.weight', 'encoder.layer.5.output.dense.bias', 'encoder.layer.1.attention.self.value

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [187]:
### Remove output folder
!rm -rf outputs

In [195]:
# Train the model
model.train_model(train, eval_data=test)

add example index and unique id: 100%|██████████| 3200/3200 [00:00<00:00, 415162.02it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Training loss,█▁
correct,▁▇███
eval_loss,█▅▃▁▁
global_step,▁▃▃▅▆▆█
incorrect,▁█▅▄▅
lr,█▁
similar,█▁▁▂▁
train_loss,█▄▂▁▂
Training loss,0.89032
correct,328
eval_loss,-4.61178


Running Epoch 0 of 5:   0%|          | 0/26 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

...

Running Epoch 3 of 5:   0%|          | 0/26 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/26 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

(130,
 {'global_step': [26, 52, 78, 104, 130],
  'correct': [0, 0, 0, 0, 1],
  'similar': [800, 800, 800, 786, 776],
  'incorrect': [0, 0, 0, 14, 23],
  'train_loss': [3.7362747192382812,
   3.6342151165008545,
   3.166215419769287,
   3.150390625,
   3.169034004211426],
  'eval_loss': [-0.27265343299278844,
   -0.16620518611027643,
   -0.6009615384615384,
   -0.3680419921875,
   -0.7717472956730769]})

In [196]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [197]:
texts

{'correct_text': {'56d5307f2593cc1400307abd': '3,000 to 5,000'},
 'similar_text': {'56bf91c6a10cfb1400551183': {'truth': 'March 2006',
   'predicted': '',
   'question': "When did Destiny's Child get their star on the Hollywood Walk of Fame?"},
  '56d4bc642ccc5a1400d83191': {'truth': '541,000',
   'predicted': '',
   'question': "How many copies did B'Day sell during the first week of its release?"},
  '56d4d18d2ccc5a1400d8325c': {'truth': 'Beyonce pregnant',
   'predicted': '',
   'question': 'What was the most searched term on Google for the week of August 29, 2011?'},
  '56bfbc17a10cfb1400551290': {'truth': 'Vision of Love',
   'predicted': '',
   'question': 'What song by Mariah Carey influenced her?'},
  '56bf95eaa10cfb1400551194': {'truth': '158.8 million',
   'predicted': '',
   'question': "How many millions of dollars did ''The Pink Panther'' gross world-wide?"},
  '56cf69144df3c31400b0d746': {'truth': 'Feliks Jarocki',
   'predicted': '',
   'question': 'With whom did Chopin 

In [198]:
context = """Cristiano Ronaldo dos Santos Aveiro GOIH ComM (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Professional League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards[note 3] and four European Golden Shoes, the most by a European player. He has won 32 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42) in the Champions League, goals in the European Championship (14), international goals (123) and international appearances (200). He is one of the few players to have made over 1,100 professional career appearances, and has scored over 800 official senior career goals for club and country, making him the highest goalscorer of all time.

Ronaldo began his senior career with Sporting CP, before signing with Manchester United in 2003, winning the FA Cup in his first season. He would also go on to win three consecutive Premier League titles, the Champions League and the FIFA Club World Cup; at age 23, he won his first Ballon d'Or. Ronaldo was the subject of the then-most expensive association football transfer when he signed for Real Madrid in 2009 in a transfer worth €94 million (£80 million). He became a key contributor and formed an attacking trio with Karim Benzema and Gareth Bale which was integral to the team winning four Champions League wins from 2014 to 2018, including La Décima. During this period, he won back-to-back Ballons d'Or in 2013 and 2014, and again in 2016 and 2017, and was runner-up three times behind Lionel Messi, his perceived career rival. He also became the club's all-time top goalscorer and the all-time top scorer in the Champions League, and finished as the competition's top scorer for six consecutive seasons between 2012 and 2018. With Real, Ronaldo won four Champions Leagues, two La Liga titles, two Copas del Rey, three UEFA Super Cups and three Club World Cups. In 2018, he signed for Juventus in a transfer worth an initial €100 million (£88 million), the most expensive transfer for an Italian club and for a player over 30 years old. He won two Serie A titles, two Supercoppa Italiana trophies and a Coppa Italia, became the inaugural Serie A Most Valuable Player and became the first footballer to finish as top scorer in the English, Spanish and Italian leagues. He returned to Manchester United in 2021, finishing his only full season as the club's top scorer, before his contract was terminated in 2022. In 2023, he signed for Al Nassr.

Ronaldo made his international debut for Portugal in 2003 at the age of 18 and has since earned 200 caps, making him Portugal's most-capped player. With more than 100 goals at international level, he is also the all-time top goalscorer. Ronaldo has played in and scored at eleven major tournaments; he scored his first international goal at Euro 2004, where he helped Portugal reach the final. He assumed captaincy of the national team in July 2008. In 2015, Ronaldo was named the best Portuguese player of all time by the Portuguese Football Federation. The following year, he led Portugal to their first major tournament title at Euro 2016, and received the Silver Boot as the second-highest goalscorer of the tournament. This achievement would see him receive his fourth Ballon d'Or. He also led them to victory in the inaugural UEFA Nations League in 2019, receiving the top scorer award in the finals, and later received the Golden Boot as top scorer of Euro 2020.

One of the world's most marketable and famous athletes, Ronaldo was ranked the world's highest-paid athlete by Forbes in 2016 and 2017 and the world's most famous athlete by ESPN from 2016 to 2019. Time included him on their list of the 100 most influential people in the world in 2014. He is the first footballer and the third sportsman to earn US$1 billion in his career."""

In [201]:
# Make predictions with the model
to_predict = [
    {
        "context": context,
        "qas": [
            {
                "question": "Who is Ronaldo?",
                "id": "0",
            }
        ],
    }
]

In [202]:
answers, probabilities = model.predict(to_predict)

print(answers), probabilities

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2906.66it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['', '2013 and 2014, and again in 2016']}]


(None, [{'id': '0', 'probability': [0.7025221250285169, 0.1586016393319683]}])

# electra small

In [208]:
model_type == "electra"
model_name = "google/electra-small-discriminator"

In [209]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "use_early_stopping": True,
    'early_stopping_consider_epochs': False,
    'early_stopping_delta': 0,
    # 'early_stopping_metric': f"str = {'correct'}",
    'early_stopping_metric_minimize': False,
    'early_stopping_patience': 2,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }

}

In [210]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['encoder.layer.9.output.dense.weight', 'encoder.layer.1.attention.output.dense.weight', 'embeddings.LayerNorm.weight', 'encoder.layer.6.attention.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.8.output.dense.bias', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.1.attention.self.query.weight', 'encoder.layer.1.attention.self.query.bias', 'encoder.layer.3.attention.output.LayerNorm.bias', 'encoder.layer.2.attention.self.key.bias', 'embeddings.word_embeddings.weight', 'encoder.layer.5.output.LayerNorm.weight', 'encoder.layer.9.attention.self.key.weight', 'encoder.layer.9.attention.self.key.bias', 'encoder.layer.9.attention.output.dense.weight', 'encoder.layer.5.attention.output.dense.weight', 'encoder.layer.5.output.dense.bias', 'encoder.layer.1.attenti

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [206]:
### Remove output folder
!rm -rf outputs

In [211]:
# Train the model
model.train_model(train, eval_data=test)

add example index and unique id: 100%|██████████| 3200/3200 [00:00<00:00, 508458.26it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Training loss,█▁
correct,▁▁▁▁█
eval_loss,▇█▃▆▁
global_step,▁▃▃▅▆▆█
incorrect,▁▁▁▅█
lr,█▁
similar,███▄▁
train_loss,█▇▁▁▁
Training loss,3.43139
correct,1
eval_loss,-0.77175


Running Epoch 0 of 5:   0%|          | 0/26 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

...

Running Epoch 3 of 5:   0%|          | 0/26 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/26 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

(130,
 {'global_step': [26, 52, 78, 104, 130],
  'correct': [0, 0, 0, 0, 0],
  'similar': [800, 800, 800, 800, 800],
  'incorrect': [0, 0, 0, 0, 0],
  'train_loss': [4.339188098907471,
   4.401088237762451,
   3.8810129165649414,
   3.6708426475524902,
   3.503654956817627],
  'eval_loss': [-1.3719951923076923,
   -0.708984375,
   -0.4498948317307692,
   -0.3217210036057692,
   -0.33993765024038464]})

In [212]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [213]:
texts

{'correct_text': {},
 'similar_text': {'56bf91c6a10cfb1400551183': {'truth': 'March 2006',
   'predicted': '',
   'question': "When did Destiny's Child get their star on the Hollywood Walk of Fame?"},
  '56d4bc642ccc5a1400d83191': {'truth': '541,000',
   'predicted': '',
   'question': "How many copies did B'Day sell during the first week of its release?"},
  '56d4d18d2ccc5a1400d8325c': {'truth': 'Beyonce pregnant',
   'predicted': '',
   'question': 'What was the most searched term on Google for the week of August 29, 2011?'},
  '56bfbc17a10cfb1400551290': {'truth': 'Vision of Love',
   'predicted': '',
   'question': 'What song by Mariah Carey influenced her?'},
  '56bf95eaa10cfb1400551194': {'truth': '158.8 million',
   'predicted': '',
   'question': "How many millions of dollars did ''The Pink Panther'' gross world-wide?"},
  '56cf69144df3c31400b0d746': {'truth': 'Feliks Jarocki',
   'predicted': '',
   'question': 'With whom did Chopin go to Berlin?'},
  '56cbdcd16d243a140015eda2

In [214]:
context = """Cristiano Ronaldo dos Santos Aveiro GOIH ComM (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Professional League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards[note 3] and four European Golden Shoes, the most by a European player. He has won 32 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42) in the Champions League, goals in the European Championship (14), international goals (123) and international appearances (200). He is one of the few players to have made over 1,100 professional career appearances, and has scored over 800 official senior career goals for club and country, making him the highest goalscorer of all time.

Ronaldo began his senior career with Sporting CP, before signing with Manchester United in 2003, winning the FA Cup in his first season. He would also go on to win three consecutive Premier League titles, the Champions League and the FIFA Club World Cup; at age 23, he won his first Ballon d'Or. Ronaldo was the subject of the then-most expensive association football transfer when he signed for Real Madrid in 2009 in a transfer worth €94 million (£80 million). He became a key contributor and formed an attacking trio with Karim Benzema and Gareth Bale which was integral to the team winning four Champions League wins from 2014 to 2018, including La Décima. During this period, he won back-to-back Ballons d'Or in 2013 and 2014, and again in 2016 and 2017, and was runner-up three times behind Lionel Messi, his perceived career rival. He also became the club's all-time top goalscorer and the all-time top scorer in the Champions League, and finished as the competition's top scorer for six consecutive seasons between 2012 and 2018. With Real, Ronaldo won four Champions Leagues, two La Liga titles, two Copas del Rey, three UEFA Super Cups and three Club World Cups. In 2018, he signed for Juventus in a transfer worth an initial €100 million (£88 million), the most expensive transfer for an Italian club and for a player over 30 years old. He won two Serie A titles, two Supercoppa Italiana trophies and a Coppa Italia, became the inaugural Serie A Most Valuable Player and became the first footballer to finish as top scorer in the English, Spanish and Italian leagues. He returned to Manchester United in 2021, finishing his only full season as the club's top scorer, before his contract was terminated in 2022. In 2023, he signed for Al Nassr.

Ronaldo made his international debut for Portugal in 2003 at the age of 18 and has since earned 200 caps, making him Portugal's most-capped player. With more than 100 goals at international level, he is also the all-time top goalscorer. Ronaldo has played in and scored at eleven major tournaments; he scored his first international goal at Euro 2004, where he helped Portugal reach the final. He assumed captaincy of the national team in July 2008. In 2015, Ronaldo was named the best Portuguese player of all time by the Portuguese Football Federation. The following year, he led Portugal to their first major tournament title at Euro 2016, and received the Silver Boot as the second-highest goalscorer of the tournament. This achievement would see him receive his fourth Ballon d'Or. He also led them to victory in the inaugural UEFA Nations League in 2019, receiving the top scorer award in the finals, and later received the Golden Boot as top scorer of Euro 2020.

One of the world's most marketable and famous athletes, Ronaldo was ranked the world's highest-paid athlete by Forbes in 2016 and 2017 and the world's most famous athlete by ESPN from 2016 to 2019. Time included him on their list of the 100 most influential people in the world in 2014. He is the first footballer and the third sportsman to earn US$1 billion in his career."""

In [215]:
# Make predictions with the model
to_predict = [
    {
        "context": context,
        "qas": [
            {
                "question": "Who is Ronaldo?",
                "id": "0",
            }
        ],
    }
]

In [216]:
answers, probabilities = model.predict(to_predict)

print(answers), probabilities

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3175.10it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['', 'Real, Ronaldo won four Champions Leagues, two La Liga titles, two Copas del Rey, three UEFA Super Cups and three Club World Cups. In 2018, he signed for Juventus in a transfer worth an initial €100 million']}]


(None, [{'id': '0', 'probability': [0.8862887768285698, 0.05943566061079244]}])

# Final Model

In [229]:
### Advanced Methodology

model_type="bert"
model_name= "bert-base-cased"

### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "use_early_stopping": True,
    'early_stopping_consider_epochs': False,
    'early_stopping_delta': 0,
    # 'early_stopping_metric': f"str = {'correct'}",
    'early_stopping_metric_minimize': False,
    'early_stopping_patience': 2,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }

}

In [230]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [231]:
### Remove output folder
!rm -rf outputs

In [232]:
# Train the model
model.train_model(train, eval_data=test)

add example index and unique id: 100%|██████████| 8000/8000 [00:00<00:00, 696048.96it/s]


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Training loss,▁
correct,▁
eval_loss,▁
global_step,▁█
incorrect,▁
lr,▁
similar,▁
train_loss,▁
Training loss,2.62052
correct,535
eval_loss,-2.74988


Running Epoch 0 of 10:   0%|          | 0/64 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

...

Running Epoch 8 of 10:   0%|          | 0/64 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/64 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

(640,
 {'global_step': [64, 128, 192, 256, 320, 384, 448, 512, 576, 640],
  'correct': [617, 856, 915, 914, 908, 887, 885, 893, 891, 894],
  'similar': [831, 806, 786, 774, 742, 830, 793, 770, 789, 793],
  'incorrect': [552, 338, 299, 312, 350, 283, 322, 337, 320, 313],
  'train_loss': [1.9576486349105835,
   1.4539179801940918,
   0.8765764236450195,
   0.6072497367858887,
   0.6027660369873047,
   0.20754477381706238,
   0.30075883865356445,
   0.21144582331180573,
   0.2595638632774353,
   0.1361563801765442],
  'eval_loss': [-2.46063232421875,
   -3.98876953125,
   -5.273681640625,
   -5.888427734375,
   -6.67626953125,
   -7.1942138671875,
   -7.360107421875,
   -7.444091796875,
   -7.53076171875,
   -7.6033935546875]})

In [233]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

In [234]:
texts

{'correct_text': {'56ce30f5aab44d1400b88521': 'Pulitzer Prize',
  '5733a866d058e614000b5f95': 'United States',
  '573417154776f41900661845': '10,562,178',
  '56cf6af94df3c31400b0d761': 'Souvenir de Paganini',
  '56d976addc89441400fdb4bf': "Macau Fisherman's Wharf",
  '56d9a582dc89441400fdb68a': '63 days',
  '56d1e9ace7d4791d009024b2': 'śūnyatā',
  '56cfe547234ae51400d9c02a': '1,575–49,837 EJ per year',
  '5733f37ed058e614000b664f': 'three',
  '56cc55fa6d243a140015ef15': '2 GB',
  '56d37eef59d6e4140014654c': 'five',
  '56daf8e2e7c41114004b4bdc': 'four',
  '56ce7ae2aab44d1400b887e9': 'Kane Kramer',
  '56d3765559d6e41400146480': 'over 100,000',
  '56db2011e7c41114004b4dac': 'Jennifer Lopez',
  '5733b2e14776f41900661085': 'Archaeology',
  '5730976a069b5314008321bb': 'Henry Nelson Wieman',
  '573260db0fdd8d15006c6a67': 'major global financial institutions',
  '57341a094776f41900661875': '6.8%',
  '56d1d3fee7d4791d00902264': 'Lorraine',
  '56db7278e7c41114004b5125': 'Reporters Without Border

In [235]:
  context = """Cristiano Ronaldo dos Santos Aveiro GOIH ComM (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Professional League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards[note 3] and four European Golden Shoes, the most by a European player. He has won 32 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42) in the Champions League, goals in the European Championship (14), international goals (123) and international appearances (200). He is one of the few players to have made over 1,100 professional career appearances, and has scored over 800 official senior career goals for club and country, making him the highest goalscorer of all time.

  Ronaldo began his senior career with Sporting CP, before signing with Manchester United in 2003, winning the FA Cup in his first season. He would also go on to win three consecutive Premier League titles, the Champions League and the FIFA Club World Cup; at age 23, he won his first Ballon d'Or. Ronaldo was the subject of the then-most expensive association football transfer when he signed for Real Madrid in 2009 in a transfer worth €94 million (£80 million). He became a key contributor and formed an attacking trio with Karim Benzema and Gareth Bale which was integral to the team winning four Champions League wins from 2014 to 2018, including La Décima. During this period, he won back-to-back Ballons d'Or in 2013 and 2014, and again in 2016 and 2017, and was runner-up three times behind Lionel Messi, his perceived career rival. He also became the club's all-time top goalscorer and the all-time top scorer in the Champions League, and finished as the competition's top scorer for six consecutive seasons between 2012 and 2018. With Real, Ronaldo won four Champions Leagues, two La Liga titles, two Copas del Rey, three UEFA Super Cups and three Club World Cups. In 2018, he signed for Juventus in a transfer worth an initial €100 million (£88 million), the most expensive transfer for an Italian club and for a player over 30 years old. He won two Serie A titles, two Supercoppa Italiana trophies and a Coppa Italia, became the inaugural Serie A Most Valuable Player and became the first footballer to finish as top scorer in the English, Spanish and Italian leagues. He returned to Manchester United in 2021, finishing his only full season as the club's top scorer, before his contract was terminated in 2022. In 2023, he signed for Al Nassr.

  Ronaldo made his international debut for Portugal in 2003 at the age of 18 and has since earned 200 caps, making him Portugal's most-capped player. With more than 100 goals at international level, he is also the all-time top goalscorer. Ronaldo has played in and scored at eleven major tournaments; he scored his first international goal at Euro 2004, where he helped Portugal reach the final. He assumed captaincy of the national team in July 2008. In 2015, Ronaldo was named the best Portuguese player of all time by the Portuguese Football Federation. The following year, he led Portugal to their first major tournament title at Euro 2016, and received the Silver Boot as the second-highest goalscorer of the tournament. This achievement would see him receive his fourth Ballon d'Or. He also led them to victory in the inaugural UEFA Nations League in 2019, receiving the top scorer award in the finals, and later received the Golden Boot as top scorer of Euro 2020.

  One of the world's most marketable and famous athletes, Ronaldo was ranked the world's highest-paid athlete by Forbes in 2016 and 2017 and the world's most famous athlete by ESPN from 2016 to 2019. Time included him on their list of the 100 most influential people in the world in 2014. He is the first footballer and the third sportsman to earn US$1 billion in his career."""

In [242]:
# Make predictions with the model
to_predict = [
    {
        "context": context,
        "qas": [
            {
                "question": "When did Ronaldo win Ballon d'Or?",
                "id": "0",
            }
        ],
    }
]

In [243]:
answers, probabilities = model.predict(to_predict)

print(answers), probabilities

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3246.37it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['five', '2013 and 2014', '2013']}]


(None,
 [{'id': '0',
   'probability': [0.9385814987603907,
    0.053470778939416916,
    0.007947721760881628]}])

In [254]:
import _pickle as cPickle

with open('model.pkl', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    cPickle.dump(model, f)

In [255]:
with open('model.pkl', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    Model = cPickle.load(f)

In [256]:
answers, probabilities = Model.predict(to_predict)

print(answers), probabilities

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3279.36it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['five', '2013 and 2014', '2013']}]


(None,
 [{'id': '0',
   'probability': [0.9385814987603907,
    0.053470778939416916,
    0.007947721760881628]}])